## ref : https://blog.naver.com/21ahn/221329975739 (Crawling by POST method)

In [1]:
from bs4 import BeautifulSoup
import json, random
import requests

## 1. Define functions

In [2]:
# function1. generate sentece
def make_sentence(dic):
    ret = []
    if not "@" in dic:
        return "not an appropraite json format"
    
    beginner_dic = dic["@"]
    w1 = word_choice(beginner_dic)
    w2 = word_choice(beginner_dic[w1])
    ret.append(w1)
    ret.append(w2)
    
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 in (".", "\n"):
            break
        w1, w2 = w2, w3
    ret = "".join(ret)
    ret_daum_corrected = kor_corrector_daum(ret)
    # ret_pusan_corrected = kor_corrector_pusan(ret)  # deleted : Not properly runnging
    
    r_tuple = (ret, ret_daum_corrected)
    
    return r_tuple

# function1-1. randomly choose one key from the input dictionary
def word_choice(dict1):
    keys = list(dict1.keys())
    return random.choice(keys)

# function1-2. correct grammar using the Daum grammar checker
def kor_corrector_daum(text):
    main_url = 'https://alldic.daum.net/grammar_checker.do'
    form_data = {'sentence':text}

    response = requests.post(main_url, data=form_data)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        a_tag = soup.select_one('#resultForm > div.cont_grammar > div > a')
        r_text = a_tag.attrs['data-error-output']
        
    except Exception as e:
        r_text = f'Error - {str(e)}  |  status_code : {response.status_code}'
    
    return r_text

## 2. Load json data

In [3]:
dict_file = "./200205_markov-BEXX0014.json"
dic = json.load(open(dict_file))

## 3. Generate random sentences using json data

In [4]:
for i in range(3):
    s1, s2 = make_sentence(dic)
    print('original text :', s1, end='\n\n')
    print('corrected by DAUM :', s2)
    print("\n---\n")

original text : 뿐이겠소이동진선생의심리상태를그는네댓명의여학생이서있었다.

corrected by DAUM : 뿐이겠소 이동진 선생의 심리상태를 그는 네댓 명의 여학생이 서 있었다.

---

original text : 미버라해쌓든서울네까지감싸주는것을보면거의매일이다시피미군항공모함이격침됐다는기사가실려있는데전쟁은아직도끝나지않았어


corrected by DAUM : 미버라해쌓든서울네까지감싸주는 것을 보면 거의 매일이다시피미군 항공모함이 격침됐다는 기사가 실려있는데 전쟁은 아직도 끝나지 않았어

---

original text : 징용자를피해서배를탔다는것인데선구에미주시이소


corrected by DAUM : 징용자를 피해서 배를 탔다는 것인데 선구에 미주시세요

---

